# Segmenting and Clustering Neighborhoods in Toronto – Week 2 – Task 1

In [1]:
# Importing Packages
import folium  # map rendering library
### import json  # library to handle JSON files
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup  # for extracting data from HTML and XML files Library
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values
from pandas import json_normalize  # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### URL from wikipedia - List of postal codes of Canada: M

This is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario. Only the first three characters are listed, corresponding to the Forward Sortation Area.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
toronto_wikipedia_data = requests.get(url)

In [5]:
print(toronto_wikipedia_data)

<Response [200]>


In [6]:
print(toronto_wikipedia_data.status_code)

200


## We will follow the guidelines of the problem statement

### Tips for  Webscraping Updated Table in Week3 Peer Graded Assignment

 **After retreiving the URL and creating a Beautiful soup object** 

 **Firstly create a list**  

 **Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.**

 **As postal code contains upto 3 characters extract that using tablerow.p.text**

 **Next use split ,strip and replace functions for getting Borough and Neighborhood information.**.

 **Append to the list**  

 **Create a dataframe with list**

In [7]:
soup = BeautifulSoup(toronto_wikipedia_data.content, 'html.parser')

In [8]:
tables = soup.find('table')

In [9]:
trs = tables.find_all('tr')

In [10]:
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)

##### Following instructions:

Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'.

In [11]:
trb = []
for row in rows:
    for i in range(9):
        trab = str(row[i].text.strip(' \n'))
        if trab[3:] != 'Not assigned':
            postalcode = trab[:3]
            aaa = ''
            bol1 = True
            for j in range(3, len(trab)):
                if trab[j] == '(':
                    bol1 = True
                elif trab[j] == ')':
                    aaa = aaa + trab[j]
                    bol1 = False
                if bol1:
                    aaa = aaa + trab[j]

            borough = ''
            neighborhood = ''
            bol1 = True
            for j in range(len(aaa)):
                if aaa[j] == '(':
                    bol1 = False
                if bol1:
                    borough = borough + aaa[j]
                else:
                    neighborhood = neighborhood + aaa[j]
            borough = str(str(borough.strip()).replace('YorkEast','York East')).replace('Stn A PO Boxes25 The Esplanade','')
            borough = str(borough).replace('Canada Post Gateway Processing Centre','')
            borough = str(borough).replace('Business reply mail Processing Centre969 Eastern','')
            borough = str(borough).replace('EtobicokeNorthwest','Etobicoke Northwest')
            neighborhood = str(neighborhood.strip(' ()')).replace('/', ',').replace('(',' ').replace(')',',')
            trb.append([postalcode, borough, neighborhood])

In [12]:
# Let's check the trb list before load into a pandas dataframe
trb[:10]

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park , Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor , Lawrence Heights'],
 ['M7A', "Queen's Park", 'Ontario Provincial Government'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Malvern , Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill , Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson']]

##### Let's tranform the loaded data into a pandas dataframe

In [13]:
# define the dataframe columns
df_column_names = ['Postal Code', 'Borough', 'Neighborhood']

In [14]:
toronto_df = pd.DataFrame(trb, columns=df_column_names)

In [15]:
toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Postal Code   103 non-null    object
 1   Borough       103 non-null    object
 2   Neighborhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [16]:
print(toronto_df.shape)

(103, 3)


In [17]:
toronto_df.head(15)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [18]:
toronto_df = toronto_df.sort_values(by=['Postal Code', 'Borough'], ascending=True).reset_index()

In [19]:
toronto_df.head(15)

,index,Postal Code,Borough,Neighborhood
0,6,M1B,Scarborough,"Malvern , Rouge"
1,12,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,18,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,22,M1G,Scarborough,Woburn
4,26,M1H,Scarborough,Cedarbrae
5,32,M1J,Scarborough,Scarborough Village
6,38,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,44,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,51,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,58,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [20]:
toronto_df.tail(15)

,index,Postal Code,Borough,Neighborhood
88,88,M8V,Etobicoke,"New Toronto , Mimico South , Humber Bay Shores"
89,93,M8W,Etobicoke,"Alderwood , Long Branch"
90,98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
91,101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."
92,102,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo..."
93,5,M9A,Etobicoke,Islington Avenue
94,11,M9B,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr..."
95,17,M9C,Etobicoke,"Eringate , Bloordale Gardens , Old Burnhamthor..."
96,50,M9L,North York,Humber Summit
97,57,M9M,North York,"Humberlea , Emery"


Reading the downloaded file Geospatial_Coordinates.csv

In [21]:
toronto_geo = pd.read_csv('Geospatial_Coordinates.csv', sep=',')

In [22]:
toronto_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### joining the 2 dataframes - combined the Postal Code Column

In [23]:
toronto_df = pd.merge(toronto_df, toronto_geo, on="Postal Code", how='left')

In [24]:
toronto_df.head(15)

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,12,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,18,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,22,M1G,Scarborough,Woburn,43.770992,-79.216917
4,26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,38,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,44,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,51,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,58,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


In [25]:
toronto_df.tail(15)

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
88,88,M8V,Etobicoke,"New Toronto , Mimico South , Humber Bay Shores",43.605647,-79.501321
89,93,M8W,Etobicoke,"Alderwood , Long Branch",43.602414,-79.543484
90,98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
91,101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
92,102,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.628841,-79.520999
93,5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
94,11,M9B,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.650943,-79.554724
95,17,M9C,Etobicoke,"Eringate , Bloordale Gardens , Old Burnhamthor...",43.643515,-79.577201
96,50,M9L,North York,Humber Summit,43.756303,-79.565963
97,57,M9M,North York,"Humberlea , Emery",43.724766,-79.532242


In [26]:
toronto_df.set_index('Postal Code', inplace = True)

taking the geolocation of Toronto

In [27]:
address = 'Toronto, TO'

In [28]:
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [29]:
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65238435, -79.38356765.


### Create a map of Toronto with neighborhoods superimposed on top.

In [30]:
# create a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [31]:
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

In [32]:
map_toronto

### Let's Explore the Toronto Boroughs

##### 1. Exploring Scarborough

In [33]:
Scarborough_data = toronto_df[toronto_df['Borough'] == 'Scarborough'].reset_index(drop=True)

In [34]:
Scarborough_data.head(10)

,index,Borough,Neighborhood,Latitude,Longitude
0,6,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,12,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,18,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,22,Scarborough,Woburn,43.770992,-79.216917
4,26,Scarborough,Cedarbrae,43.773136,-79.239476
5,32,Scarborough,Scarborough Village,43.744734,-79.239476
6,38,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,44,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,51,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,58,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


###### Get the geolocator of Scarborough

In [35]:
address = 'Scarborough, SB'

In [36]:
geolocator = Nominatim(user_agent="sb_explorer")

In [37]:
location = geolocator.geocode(address)

In [38]:
latitude = location.latitude
longitude = location.longitude

In [39]:
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7169869, -79.2546806.


In [40]:
# Create a map of Scarborough using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

In [41]:
# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)

In [42]:
map_Scarborough

##### 2. Exploring Downtown Toronto

In [43]:
Downtown_Toronto_data = toronto_df[toronto_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

In [44]:
Downtown_Toronto_data.head(10)

,index,Borough,Neighborhood,Latitude,Longitude
0,91,Downtown Toronto,Rosedale,43.679563,-79.377529
1,96,Downtown Toronto,"St. James Town , Cabbagetown",43.667967,-79.367675
2,99,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,2,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,15,Downtown Toronto,St. James Town,43.651494,-79.375418
6,20,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,24,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,30,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
9,36,Downtown Toronto,"Harbourfront East , Union Station , Toronto Is...",43.640816,-79.381752


###### Get the geolocator of Downtown Toronto

In [45]:
address = 'Downtown Toronto, DT'

In [46]:
geolocator = Nominatim(user_agent="dt_explorer")

In [47]:
location = geolocator.geocode(address)

In [48]:
latitude = location.latitude
longitude = location.longitude

In [49]:
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081162653639.


In [50]:
# Create a map of Downtown Toronto using latitude and longitude values
map_Downtown_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [51]:
# add markers to map
for lat, lng, label in zip(Downtown_Toronto_data['Latitude'], Downtown_Toronto_data['Longitude'], Downtown_Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)

In [52]:
map_Downtown_Toronto

##### 3. Exploring Central Toronto

In [53]:
Central_Toronto_data = toronto_df[toronto_df['Borough'] == 'Central Toronto'].reset_index(drop=True)

In [54]:
Central_Toronto_data.head(10)

,index,Borough,Neighborhood,Latitude,Longitude
0,61,Central Toronto,Lawrence Park,43.728020,-79.388790
1,67,Central Toronto,Davisville North,43.712751,-79.390197
2,73,Central Toronto,North Toronto West,43.715383,-79.405678
3,79,Central Toronto,Davisville,43.704324,-79.388790
4,83,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160
5,86,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686412,-79.400049
6,62,Central Toronto,Roselawn,43.711695,-79.416936
7,68,Central Toronto,Forest Hill North & West,43.696948,-79.411307
8,74,Central Toronto,"The Annex , North Midtown , Yorkville",43.672710,-79.405678


###### Get the geolocator of Central Toronto

In [55]:
address = 'Central Toronto, CT'

In [56]:
geolocator = Nominatim(user_agent="ct_explorer")

In [57]:
location = geolocator.geocode(address)

In [58]:
latitude = location.latitude
longitude = location.longitude

In [59]:
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


In [60]:
# Create a map of Central Toronto using latitude and longitude values
map_Central_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [61]:
# add markers to map
for lat, lng, label in zip(Central_Toronto_data['Latitude'], Central_Toronto_data['Longitude'], Central_Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Central_Toronto)

In [62]:
map_Central_Toronto

##### 4. Exploring Etobicoke

In [63]:
Etobicoke_data = toronto_df[toronto_df['Borough'] == 'Etobicoke'].reset_index(drop=True)

In [64]:
Etobicoke_data.head(10)

,index,Borough,Neighborhood,Latitude,Longitude
0,88,Etobicoke,"New Toronto , Mimico South , Humber Bay Shores",43.605647,-79.501321
1,93,Etobicoke,"Alderwood , Long Branch",43.602414,-79.543484
2,98,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
3,101,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
4,102,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.628841,-79.520999
5,5,Etobicoke,Islington Avenue,43.667856,-79.532242
6,11,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.650943,-79.554724
7,17,Etobicoke,"Eringate , Bloordale Gardens , Old Burnhamthor...",43.643515,-79.577201
8,70,Etobicoke,Westmount,43.696319,-79.532242
9,77,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724


###### Get the geolocator of Etobicoke

In [65]:
address = 'Etobicoke, ET'

In [66]:
geolocator = Nominatim(user_agent="et_explorer")

In [67]:
location = geolocator.geocode(address)

In [68]:
latitude = location.latitude
longitude = location.longitude

In [69]:
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [70]:
# Create a map of Etobicoke using latitude and longitude values
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=10)

In [71]:
# add markers to map
for lat, lng, label in zip(Etobicoke_data['Latitude'], Etobicoke_data['Longitude'], Etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)

In [72]:
map_Etobicoke

### Exploring and clustering venues in the neighborhoods in Toronto

In [73]:
CLIENT_ID = 'CR5KMIWNI2BGFGZSC5AOX3N5MXJLOD0RKVKFSXABZAFQ0WYW'
CLIENT_SECRET = 'NNMXIQWREEGYASC0AZZ2D1IX2PGBBD0VEGR0VYL4NFMM4UZI'
VERSION = '20180605'

In [74]:
neighborhood_latitude = toronto_df.loc['M1C']['Latitude']
neighborhood_longitude = toronto_df.loc['M1C']['Longitude']

In [75]:
LIMIT = 3500  # limit of number of venues
radius = 700  # define radius

Create URL

In [76]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [77]:
# Checking URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CR5KMIWNI2BGFGZSC5AOX3N5MXJLOD0RKVKFSXABZAFQ0WYW&client_secret=NNMXIQWREEGYASC0AZZ2D1IX2PGBBD0VEGR0VYL4NFMM4UZI&v=20180605&ll=43.7845351,-79.1604971&radius=700&limit=3500'

In [78]:
results = requests.get(url).json()

In [79]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
venues = results['response']['groups'][0]['items']

In [81]:
nearby_venues = json_normalize(venues) # flatten JSON

In [82]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

In [83]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

In [84]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [85]:
nearby_venues.shape[0]

4

In [86]:
nearby_venues

,name,categories,lat,lng
0,Shamrock Burgers,Burger Joint,43.783823,-79.168406
1,Ted's Restaurant,Breakfast Spot,43.784468,-79.169200
2,Royal Canadian Legion,Bar,43.782533,-79.163085
3,Amigo's,Breakfast Spot,43.783749,-79.168691


### Let's Explore Neighborhoods in Toronto

In [87]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # print(url)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',
                  'Venue Category',
                  'Venue Latitude', 
                  'Venue Longitude']
    
    return(nearby_venues)

In [88]:
toronto_venues = getNearbyVenues(names=toronto_df['Borough'],latitudes=toronto_df['Latitude'],longitudes=toronto_df['Longitude'])

In [89]:
toronto_venues.shape

(4878, 7)

In [90]:
toronto_venues.head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,Scarborough,43.806686,-79.194353,Images Salon & Spa,Spa,43.802283,-79.198565
1,Scarborough,43.806686,-79.194353,Harvey's,Restaurant,43.800020,-79.198307
2,Scarborough,43.806686,-79.194353,RBC Royal Bank,Bank,43.798782,-79.197090
3,Scarborough,43.806686,-79.194353,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,Scarborough,43.806686,-79.194353,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
5,Scarborough,43.806686,-79.194353,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
6,Scarborough,43.806686,-79.194353,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
7,Scarborough,43.806686,-79.194353,Tim Hortons,Coffee Shop,43.802000,-79.198169
8,Scarborough,43.806686,-79.194353,Lee Valley,Hobby Shop,43.803161,-79.199681
9,Scarborough,43.806686,-79.194353,Scarboro Garage Doors Ltd.,Home Service,43.807923,-79.201800


In [91]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
Neighborhood,,,,,,
Central Toronto,568,568,568,568,568,568
Downtown Toronto,1573,1573,1573,1573,1573,1573
East Toronto,401,401,401,401,401,401
East York,157,157,157,157,157,157
East York East Toronto,91,91,91,91,91,91
Etobicoke,252,252,252,252,252,252
Etobicoke Northwest,1,1,1,1,1,1
Mississauga,56,56,56,56,56,56
North York,623,623,623,623,623,623


#### Let's find out how many unique categories can be curated from all the returned venues

In [92]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 329 uniques categories.


### Analyze Each Neighborhood

In [93]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [94]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

In [95]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [96]:
toronto_onehot.shape

(4878, 329)

In [97]:
toronto_onehot.head(15)

,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Pla

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [98]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [99]:
toronto_grouped

,Neighborhood,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Pla

In [100]:
toronto_grouped.shape

(13, 329)

#### Let's print each neighborhood along with the top 20 most common venues

In [101]:
num_top_venues = 20

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
                 venue  freq
0          Coffee Shop  0.08
1   Italian Restaurant  0.06
2                 Café  0.05
3     Sushi Restaurant  0.04
4           Restaurant  0.03
5          Pizza Place  0.03
6                 Park  0.03
7                  Gym  0.03
8                 Bank  0.02
9             Pharmacy  0.02
10     Thai Restaurant  0.02
11              Bakery  0.02
12      Sandwich Place  0.02
13       Grocery Store  0.02
14               Diner  0.01
15         Salad Place  0.01
16    Ramen Restaurant  0.01
17                 Pub  0.01
18       Deli / Bodega  0.01
19        Dessert Shop  0.01


----Downtown Toronto----
                            venue  freq
0                     Coffee Shop  0.09
1                            Café  0.07
2             Japanese Restaurant  0.03
3                            Park  0.03
4                           Hotel  0.03
5                     Pizza Place  0.02
6              Seafood Restaurant  0.02
7                   

#### Let's put that into a _pandas_ dataframe

First, let's write a function to sort the venues in descending order.

In [102]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

let's create the new dataframe

In [103]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

In [104]:
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [105]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

In [106]:
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

##### display the top 20 venues for each neighborhood

In [107]:
neighborhoods_venues_sorted.head(15)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Central Toronto,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Restaurant,Park,Gym,Pizza Place,Bank,Sandwich Place,Thai Restaurant,Bakery,Grocery Store,Pharmacy,Playground,Japanese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Bookstore,Pub
1,Downtown Toronto,Coffee Shop,Café,Japanese Restaurant,Park,Hotel,Restaurant,Theater,Gastropub,Pizza Place,Seafood Restaurant,Vegetarian / Vegan Restaurant,Cosmetics Shop,Italian Restaurant,Bakery,Art Gallery,Gym,Sushi Restaurant,Thai Restaurant,Pub,Beer Bar
2,East Toronto,Coffee Shop,Café,Bakery,Pub,Greek Restaurant,Park,Brewery,Italian Restaurant,Pizza Place,Grocery Store,Diner,Indian Restaurant,Bar,American Restaurant,Restaurant,Beach,Fast Food Restaurant,Sandwich Place,Breakfast Spot,Ice Cream Shop
3,East York,Coffee Shop,Brewery,Sandwich Place,Grocery Store,Pizza Place,Park,Bank,Skating Rink,Indian Restaurant,Burger Joint,Furniture / Home Store,Restaurant,Beer Store,BBQ Joint,Electronics Store,Athletics & Sports,Café,Sporting Goods Shop,Supermarket,Shopping Mall
4,East York East Toronto,Café,Greek Restaurant,Coffee Shop,Beer Bar,Ethiopian Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Bank,Bakery,Gastropub,Breakfast Spot,Sandwich Place,Spa,Grocery Store,American Restaurant,Vietnamese Restaurant,Sports Bar,Chinese Restaurant,Fish & Chips Shop
5,Etobicoke,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Restaurant,Bank,Convenience Store,Italian Restaurant,Gas Station,Café,Sandwich Place,Bakery,Shopping Mall,Discount Store,Liquor Store,Breakfast Spot,Burger Joint,Pub,Sushi Restaurant
6,Etobicoke Northwest,Coffee Shop,Zoo,Music Venue,Organic Grocery,Optical Shop,Opera House,Office,Noodle House,Nightclub,New American Restaurant,Nail Salon,Music Store,Other Repair Shop,Music School,Museum,Movie Theater,Motorcycle Shop,Moroccan Restaurant,Monument / Landmark,Modern European Restaurant
7,Mississauga,Coffee Shop,Middle Eastern Restaurant,Hotel,Bus Station,Fried Chicken Joint,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Indian Restaurant,Mexican Restaurant,Gym,Burrito Place,Portuguese Restaurant,Shopping Mall,Caribbean Restaurant,Burger Joint,Breakfast Spot,Sandwich Place,Comedy Club
8,North York,Coffee Shop,Park,Pizza Place,Restaurant,Fast Food Restaurant,Grocery Store,Bank,Sandwich Place,Japanese Restaurant,Pharmacy,Intersection,Bakery,Gas Station,Korean Restaurant,Shopping Mall,Chinese Restaurant,Italian Restaurant,Café,Supermarket,Fried Chicken Joint
9,Queen's Park,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Park,Pizza Place,Yoga Studio,Gastropub,Clothing Store,Bubble Tea Shop,Breakfast Spot,Bookstore,Dance Studio,Restaurant,Ramen Restaurant,Thai Restaurant,Café,Italian Restaurant,Middle Eastern Restaurant,Salon / Barbershop,Creperie


### Clustering Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [108]:
# set number of clusters
kclusters = 5

In [109]:
toronto_grouped.shape

(13, 329)

In [110]:
toronto_grouped.head(15)

,Neighborhood,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Pla

In [111]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [112]:
toronto_grouped_clustering.shape

(13, 328)

In [113]:
toronto_grouped_clustering.head(15)

,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Amer

In [114]:
# run k-means clustering
model = KMeans(n_clusters=kclusters, random_state=2)

In [115]:
model.fit(toronto_grouped_clustering)

KMeans(n_clusters=5, random_state=2)

In [116]:
# check cluster labels generated for each row in the dataframe
model.labels_

array([4, 4, 1, 3, 1, 3, 0, 2, 3, 4, 3, 1, 3])

In [117]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Central Toronto,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Restaurant,Park,Gym,Pizza Place,Bank,Sandwich Place,Thai Restaurant,Bakery,Grocery Store,Pharmacy,Playground,Japanese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Bookstore,Pub
1,Downtown Toronto,Coffee Shop,Café,Japanese Restaurant,Park,Hotel,Restaurant,Theater,Gastropub,Pizza Place,Seafood Restaurant,Vegetarian / Vegan Restaurant,Cosmetics Shop,Italian Restaurant,Bakery,Art Gallery,Gym,Sushi Restaurant,Thai Restaurant,Pub,Beer Bar
2,East Toronto,Coffee Shop,Café,Bakery,Pub,Greek Restaurant,Park,Brewery,Italian Restaurant,Pizza Place,Grocery Store,Diner,Indian Restaurant,Bar,American Restaurant,Restaurant,Beach,Fast Food Restaurant,Sandwich Place,Breakfast Spot,Ice Cream Shop
3,East York,Coffee Shop,Brewery,Sandwich Place,Grocery Store,Pizza Place,Park,Bank,Skating Rink,Indian Restaurant,Burger Joint,Furniture / Home Store,Restaurant,Beer Store,BBQ Joint,Electronics Store,Athletics & Sports,Café,Sporting Goods Shop,Supermarket,Shopping Mall
4,East York East Toronto,Café,Greek Restaurant,Coffee Shop,Beer Bar,Ethiopian Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Bank,Bakery,Gastropub,Breakfast Spot,Sandwich Place,Spa,Grocery Store,American Restaurant,Vietnamese Restaurant,Sports Bar,Chinese Restaurant,Fish & Chips Shop
5,Etobicoke,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Restaurant,Bank,Convenience Store,Italian Restaurant,Gas Station,Café,Sandwich Place,Bakery,Shopping Mall,Discount Store,Liquor Store,Breakfast Spot,Burger Joint,Pub,Sushi Restaurant
6,Etobicoke Northwest,Coffee Shop,Zoo,Music Venue,Organic Grocery,Optical Shop,Opera House,Office,Noodle House,Nightclub,New American Restaurant,Nail Salon,Music Store,Other Repair Shop,Music School,Museum,Movie Theater,Motorcycle Shop,Moroccan Restaurant,Monument / Landmark,Modern European Restaurant
7,Mississauga,Coffee Shop,Middle Eastern Restaurant,Hotel,Bus Station,Fried Chicken Joint,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Indian Restaurant,Mexican Restaurant,Gym,Burrito Place,Portuguese Restaurant,Shopping Mall,Caribbean Restaurant,Burger Joint,Breakfast Spot,Sandwich Place,Comedy Club
8,North York,Coffee Shop,Park,Pizza Place,Restaurant,Fast Food Restaurant,Grocery Store,Bank,Sandwich Place,Japanese Restaurant,Pharmacy,Intersection,Bakery,Gas Station,Korean Restaurant,Shopping Mall,Chinese Restaurant,Italian Restaurant,Café,Supermarket,Fried Chicken Joint
9,Queen's Park,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Park,Pizza Place,Yoga Studio,Gastropub,Clothing Store,Bubble Tea Shop,Breakfast Spot,Bookstore,Dance Studio,Restaurant,Ramen Restaurant,Thai Restaurant,Café,Italian Restaurant,Middle Eastern Restaurant,Salon / Barbershop,Creperie


###### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [118]:
# add clustering labels
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', model.labels_)

In [119]:
neighborhoods_venues_sorted

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Central Toronto,4,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Restaurant,Park,Gym,Pizza Place,Bank,Sandwich Place,Thai Restaurant,Bakery,Grocery Store,Pharmacy,Playground,Japanese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Bookstore,Pub
1,Downtown Toronto,4,Coffee Shop,Café,Japanese Restaurant,Park,Hotel,Restaurant,Theater,Gastropub,Pizza Place,Seafood Restaurant,Vegetarian / Vegan Restaurant,Cosmetics Shop,Italian Restaurant,Bakery,Art Gallery,Gym,Sushi Restaurant,Thai Restaurant,Pub,Beer Bar
2,East Toronto,1,Coffee Shop,Café,Bakery,Pub,Greek Restaurant,Park,Brewery,Italian Restaurant,Pizza Place,Grocery Store,Diner,Indian Restaurant,Bar,American Restaurant,Restaurant,Beach,Fast Food Restaurant,Sandwich Place,Breakfast Spot,Ice Cream Shop
3,East York,3,Coffee Shop,Brewery,Sandwich Place,Grocery Store,Pizza Place,Park,Bank,Skating Rink,Indian Restaurant,Burger Joint,Furniture / Home Store,Restaurant,Beer Store,BBQ Joint,Electronics Store,Athletics & Sports,Café,Sporting Goods Shop,Supermarket,Shopping Mall
4,East York East Toronto,1,Café,Greek Restaurant,Coffee Shop,Beer Bar,Ethiopian Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Bank,Bakery,Gastropub,Breakfast Spot,Sandwich Place,Spa,Grocery Store,American Restaurant,Vietnamese Restaurant,Sports Bar,Chinese Restaurant,Fish & Chips Shop
5,Etobicoke,3,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Restaurant,Bank,Convenience Store,Italian Restaurant,Gas Station,Café,Sandwich Place,Bakery,Shopping Mall,Discount Store,Liquor Store,Breakfast Spot,Burger Joint,Pub,Sushi Restaurant
6,Etobicoke Northwest,0,Coffee Shop,Zoo,Music Venue,Organic Grocery,Optical Shop,Opera House,Office,Noodle House,Nightclub,New American Restaurant,Nail Salon,Music Store,Other Repair Shop,Music School,Museum,Movie Theater,Motorcycle Shop,Moroccan Restaurant,Monument / Landmark,Modern European Restaurant
7,Mississauga,2,Coffee Shop,Middle Eastern Restaurant,Hotel,Bus Station,Fried Chicken Joint,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Indian Restaurant,Mexican Restaurant,Gym,Burrito Place,Portuguese Restaurant,Shopping Mall,Caribbean Restaurant,Burger Joint,Breakfast Spot,Sandwich Place,Comedy Club
8,North York,3,Coffee Shop,Park,Pizza Place,Restaurant,Fast Food Restaurant,Grocery Store,Bank,Sandwich Place,Japanese Restaurant,Pharmacy,Intersection,Bakery,Gas Station,Korean Restaurant,Shopping Mall,Chinese Restaurant,Italian Restaurant,Café,Supermarket,Fried Chicken Joint
9,Queen's Park,4,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Park,Pizza Place,Yoga Studio,Gastropub,Clothing Store,Bubble Tea Shop,Breakfast Spot,Bookstore,Dance Studio,Restaurant,Ramen Restaurant,Thai Restaurant,Café,Italian Restaurant,Middle Eastern Restaurant,Salon / Barbershop,Creperie


In [120]:
toronto_merged = toronto_df

In [121]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

In [122]:
toronto_merged.head()

,index,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
Postal Code,,,,,,,,,,,,,,,,,,,,,,,,,,
M1B,6,Scarborough,"Malvern , Rouge",43.806686,-79.194353,3,Coffee Shop,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Restaurant,Bank,Bakery,Park,Grocery Store,Intersection,Sandwich Place,Indian Restaurant,Caribbean Restaurant,Burger Joint,Ice Cream Shop,Shopping Mall,Asian Restaurant,Convenience Store,Discount Store
M1C,12,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,3,Coffee Shop,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Restaurant,Bank,Bakery,Park,Grocery Store,Intersection,Sandwich Place,Indian Restaurant,Caribbean Restaurant,Burger Joint,Ice Cream Shop,Shopping Mall,Asian Restaurant,Convenience Store,Discount Store
M1E,18,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,3,Coffee Shop,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Restaurant,Bank,Bakery,Park,Grocery Store,Intersection,Sandwich Place,Indian Restaurant,Caribbean Restaurant,Burger Joint,Ice Cream Shop,Shopping Mall,Asian Restaurant,Convenience Store,Discount Store
M1G,22,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Restaurant,Bank,Bakery,Park,Grocery Store,Intersection,Sandwich Place,Indian Restaurant,Caribbean Restaurant,Burger Joint,Ice Cream Shop,Shopping Mall,Asian Restaurant,Convenience Store,Discount Store
M1H,26,Scarborough,Cedarbrae,43.773136,-79.239476,3,Coffee Shop,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Restaurant,Bank,Bakery,Park,Grocery Store,Intersection,Sandwich Place,Indian Restaurant,Caribbean Restaurant,Burger Joint,Ice Cream Shop,Shopping Mall,Asian Restaurant,Convenience Store,Discount Store


### Finally, let's visualize the resulting clusters

In [123]:
address = 'Toronto, TO'

In [124]:
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#### Create a map of Toronto with resulting clusters

In [125]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

In [126]:
# set color scheme for the clusters
x = np.arange(kclusters)

ys = [5 + i + (3*(x + 1)) + (i*x)**2 for i in range(kclusters)]

colors_array = cm.gnuplot(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [127]:
# add markers to the map
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)

In [128]:
map_clusters